In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import pandas as pd
from pd_doc2vec import doc2vec



client = MongoClient()
client = MongoClient('da1.eecs.utk.edu', 27017) 
#Connects to the MongoDB, make sure youre SSH'ed into the docker

collection_list = client['twitter'].collection_names() 
#Scrapes all the collection names from the Docker

collection_list  = [x for x in collection_list if "User" in x]
# looks at only collections with "User" in its name


i =0
sentimentdata = []
for posts in (collection_list):
    i = i+1
    print("Processing", posts, i, "/", str(len(collection_list)))
    for post in client['twitter'][posts].find({'is_news': {"$exists": True}}):  
        #Extracts all the entries with a sentiment field
        post.update({"Source": str(posts)}) 
        #Updates each data with the name of the collection it comes from
        sentimentdata.append(post) 

df = pd.DataFrame(sentimentdata)
# pulls data into the Pandas DataFrame


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  if sys.path[0] == '':


Processing Users_Maria 1 / 4
Processing Users_MiscNews 2 / 4
Processing Users_Irma 3 / 4
Processing Users_Florence 4 / 4


In [2]:
print(df.is_news.value_counts())
#How many news vs non-news

False    1061
True      318
Name: is_news, dtype: int64


In [3]:
print(df.columns.values) #How many Columns we have in our dataframe


['Source' '_id' 'contributors_enabled' 'created_at' 'default_profile'
 'default_profile_image' 'description' 'entities' 'favourites_count'
 'follow_request_sent' 'followers_count' 'following' 'friends_count'
 'geo_enabled' 'has_extended_profile' 'id' 'id_str' 'is_news'
 'is_translation_enabled' 'is_translator' 'lang' 'listed_count' 'location'
 'name' 'notifications' 'profile_background_color'
 'profile_background_image_url' 'profile_background_image_url_https'
 'profile_background_tile' 'profile_banner_url' 'profile_image_url'
 'profile_image_url_https' 'profile_link_color' 'profile_location'
 'profile_sidebar_border_color' 'profile_sidebar_fill_color'
 'profile_text_color' 'profile_use_background_image' 'protected'
 'retrieved_at' 'screen_name' 'status' 'statuses_count' 'time_zone'
 'translator_type' 'url' 'utc_offset' 'verified']


In [4]:

df[["verified", "protected", 'notifications','is_news','contributors_enabled', 'following'  ]] *= 1
# converts true/false bools to int

df = df[['screen_name', 'verified','statuses_count','protected', 'notifications', 'name','listed_count','is_news' ,'lang', 'contributors_enabled','description', 'favourites_count', 'followers_count', 'friends_count','Source' ]]  
# Drops columns we dont need!

In [5]:
print(df.sample(3).head(3))
#example of 3 random row

          screen_name verified  statuses_count protected notifications  \
538     LeaveMnsLeave        1            5988         0             0   
1001  Persie_Official        1             679         0             0   
1087     realmadriden        1           39141         0             0   

                       name  listed_count is_news   lang contributors_enabled  \
538       Leave Means Leave           233       0     en                    0   
1001       Robin van Persie         18810       0     en                    0   
1087  Real Madrid C.F. 🇬🇧🇺🇸          9813       0  en-gb                    0   

                                            description  favourites_count  \
538   No deal is better than a bad deal. Help us ens...               953   
1001                                                                  465   
1087  ⚽️ Official account of Real Madrid C.F. 🏆@real...               116   

      followers_count  friends_count          Source  
538           

In [6]:
# print(df.lang.value_counts()) #theres non-eng users, maybe lets drop them for now and translate them all later!

In [7]:
# df = df.loc[df['lang'] == 'en']
# print(df.lang.value_counts()) #Double check if we dropped everything not en

In [8]:
print(df.isna().sum()) #Check for empty fields!


screen_name             0
verified                0
statuses_count          0
protected               0
notifications           0
name                    0
listed_count            0
is_news                 0
lang                    0
contributors_enabled    0
description             0
favourites_count        0
followers_count         0
friends_count           0
Source                  0
dtype: int64


In [9]:
from sklearn.model_selection import train_test_split

df = df.applymap(str)

train, test = train_test_split(df, shuffle=True, test_size=0.25)

print(train['is_news'].value_counts())
print(test['is_news'].value_counts())



0    800
1    234
Name: is_news, dtype: int64
0    261
1     84
Name: is_news, dtype: int64


In [10]:
print("Setting vairables")

# We pass the class 3 fields:
# 1. The DataFrame
# 2. The X value, the Text (Pandas Series)
# 3. The Y values, the labels that correspond to the text (Pandas Series)
#                   It can be a list of the names of the columns or one as a string

x = doc2vec(train, "description", ["is_news", "Source"])

print("Scoring Model")

#returns scores of each labels accuracy from the first column passed into 
# the Y or 3 arguments, in this example it was "main"
# this uses sklearn.metrics.f1_score, you can pass in 
x.score(verbose=True)

print("Predicting on text")

# You can predict text as follows
print(x.predict_text("I dislike old cabin cruisers."))


Setting vairables
Scoring Model


  0%|          | 0/52 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 52/52 [00:00<00:00, 263.37it/s]
/home/nwest13/Curent/Manny/pd_doc2vec.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test['results'] = self.predict(test[X])


Accuracy Score:  0.8653846153846154
Label Score:  [0.9135802469135802, 0.6956521739130435]
Predicting on text
['0', 'Users_Florence']


In [11]:
sims_dict = {'0':[], '1':[]}


def distancetocolumns(df,sims_dict, col ):  # Lets add cosine distance to all labels in sims_dict as columns 
    tqdm.pandas()  #To do progress_apply, it is pandas .apply but with a progess bar 
    
    #now lets add the label distances to each row so we can use randomforrest
    df["results"] =  df[col].progress_apply(x.predict_sims)

    sims_dict = {'0':[], '1':[]}

    for index, row in df.iterrows():
        results_list =row["results"]

        for i in [i[0] for i in results_list ]:
            if i in sims_dict:
                sims_dict[i].append([rec for rec in results_list if rec[0] == i][0][1])

    #This will create a dataframe from the dict and then it will be concated to the original
    #This will add label distances
    sims_df = pd.DataFrame.from_dict(sims_dict)
    df = df.drop('results', axis=1)
    df = df.reset_index(drop=True)
    df = pd.concat([df, sims_df], axis=1)
    df = df.reset_index(drop=True)
    return df



test = distancetocolumns(test, sims_dict, 'description')
train = distancetocolumns(train, sims_dict, 'description')

  0%|          | 0/345 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 345/345 [00:00<00:00, 1049.30it/s]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
100%|██████████| 1034/1034 [00:00<00:00, 1463.85it/s]


In [12]:
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier()

#Lets get x and y for train
train_x = train[['verified', 'statuses_count','protected', 'notifications', 'listed_count', 'contributors_enabled','favourites_count', 'followers_count','friends_count', '0','1'  ]]
train_y = train[['is_news']]

#lets get x and y for test
test_x = test[['verified', 'statuses_count','protected', 'notifications', 'listed_count', 'contributors_enabled','favourites_count', 'followers_count','friends_count', '0','1'  ]]
test_y = test[['is_news']]


clf.fit(train_x, train_y)
print(clf.score(test_x,test_y))


/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


0.8869565217391304


In [13]:
test["results"]  = clf.predict(test_x)
from sklearn.metrics import f1_score
labelaccuracy = f1_score(test["results"], test['is_news'], average=None)
print(labelaccuracy)

[0.92844037 0.73103448]


In [14]:
df.loc[df['lang'] != "en"]

,screen_name,verified,statuses_count,protected,notifications,name,listed_count,is_news,lang,contributors_enabled,description,favourites_count,followers_count,friends_count,Source
27,Pontifex_es,1,1765,0,0,Papa Francisco,14220,0,it,0,Bienvenido al Twitter oficial de Su Santidad P...,0,16773296,8,Users_Maria
29,Ricardo_Arjona,1,3775,0,0,Ricardo Arjona ®,9788,0,es,0,Los enemigos que genera la independencia valen...,45,10452265,35,Users_Maria
41,davidbisbal,1,17242,0,0,David Bisbal,18351,0,es,0,https://t.co/66b36YMdTb https://t.co/I1WB9uAO2...,24506,9965908,874,Users_Maria
50,RafaelNadal,1,2742,0,0,Rafa Nadal,23427,0,es,0,Tennis player.,88,15552898,97,Users_Maria
51,realmadrid,1,64350,0,0,Real Madrid C.F.⚽,41979,0,es,0,⚽️ Cuenta oficial del Real Madrid C.F. 🏆@realm...,24,31136628,55,Users_Maria
56,neymarjr,1,41356,0,0,Neymar Jr,51081,0,pt,0,"Filho de Deus, Pai, Feliz e Ousado !",829,41564032,745,Users_Maria
59,MariaSharapova,1,3602,0,0,Maria Sharapova,7750,0,es,0,,1712,8695875,80,Users_Maria
64,FCBarcelona_es,1,99766,0,0,FC Barcelona,22317,0,es,0,25x LaLiga 30x Copa del Rey 5x UEFA Champions ...,65,13757751,76,Users_Maria
66,instagram,1,12123,0,0,Instagram,33424,0,en-gb,0,Founded in 2010 by Kevin (https://t.co/lurvWL5...,206,36936844,4,Users_Maria
68,LuceroMexico,1,37114,0,0,Lucero,6944,0,es,0,38 años de carrera en la industria del entrete...,89,8699924,50,Users_Maria


In [15]:
df.lang.value_counts()

en       1279
es         39
pt         21
ar         10
fr          8
en-gb       6
ko          6
tr          4
de          3
it          2
nl          1
Name: lang, dtype: int64

In [16]:
df.shape

(1379, 15)